# ニコニコAIスクール 第2回 機械学習入門 実践演習

## 実践演習の進め方
1. 講師が題材及びコードの説明をします
2. "WRITE ME!"の部分のコードを書いてみましょう
3. 書き始める前に必要な処理の概略を頭の中やノートに浮かべてからコードに落とし込みましょう

## 課題1. k-NN法の実装
1. まず簡単なアルゴリズムの説明をします
2. 次に与えられたデータを眺めてみましょう(実際のデータ分析でも，分析の前にデータを眺めておくことは重要です)
3. k-NN法を実装しましょう


In [ ]:
### おまじない
%matplotlib inline
### 必要なライブラリをインポート
import numpy as np
from sklearn import datasets, metrics
from matplotlib import pyplot as plt
from matplotlib import cm

### 1-1. データの観察

今回使うデータセットはdigitsデータセットです。  
Scikit-learn (第4回で正式に登場します) というライブラリが用意しているごく小規模のデータセットの1つです。  
1797枚の小さい (8x8) 手書き数字画像 (0~9)が格納されています。  
http://scikit-learn.org/stable/auto_examples/datasets/plot_digits_last_image.html

In [ ]:
# データ取得
digits = datasets.load_digits()
# 入力データ
X = digits.data
# ラベルデータ
y = digits.target

In [ ]:
#　データを色々観察してみましょう
# WRITE ME!
# Q1: 入力データの形状は？
print(X.shape)
# Q2: ラベルの形状は？
print(y.shape)
# Q3: ラベルの種類数は？ (np.uniqueを使う)
print(len(np.unique(y)))

### 画像を可視化してみましょう 

In [ ]:
# dataに画像を，labelにラベルを渡す
def visualize(data, label, nb_vis):
  data, label = data[:nb_vis], label[:nb_vis]
  if len(data) != len(label):
    print("画像とラベルの数が合いません")
    return
  
  data, label = data.astype(np.int), label.astype(np.int)
    
  num_data = len(data)
  size_window = np.ceil(np.sqrt(len(data)))
    
  samples = np.array(list(zip(data, label)))
  for index, (d, l) in enumerate(samples):
    # 画像データを格子状に配置する
    plt.subplot(size_window, size_window, index + 1)
    # 軸に関する表示はいらない
    plt.axis('off')
    # データを 8x8 のグレースケール画像として表示する
    plt.imshow(d.reshape(8, 8), cmap=cm.gray_r, interpolation='nearest')
    # 画像データのタイトルに正解ラベルを表示する
    plt.title(l, color='red')
        
  plt.show()

In [ ]:
visualize(X, y, 16)

ちなみに、人が見るためにここでは8x8の2次元配列として表しましたが、今回のk-NN法ではこれを**1列の数字の列と考えて64次元のベクトル**として扱います。  
画像としての構造を維持するタイプの学習器としては畳み込みニューラルネットワーク (CNN) などがあります (詳しくは第7回で)。

## 1-2. データ分割
講義で教わったように、機械学習では最低でも
* 訓練データ
* 検証データ (パラメータ調整用：今回は近傍数k)
* テストデータ (最終評価用)

の3つに分割する必要があります。  
交差検証は大変なので、今回は簡単なhold-out validationでデータを分割しましょう。

**Q: データ (X) をランダムにシャッフルし、適当に訓練データ・検証データ・テストデータに分割せよ。**  
変数名は次を使用すること：
* train_X, train_y
* valid_X, valid_y
* test_X, test_y

### ヒント
* シャッフルした先頭からtrain個、valid個、test個を持ってくれば良い。

In [ ]:
# 準備
X = digits.data.copy()
y = digits.target.copy()
np.random.seed(1701)

# データのシャッフル
# X, yを同じようにシャッフルするので、0〜データ数-1のインデックスをシャッフルする
indexes = np.arange(0, len(X), 1)
np.random.shuffle(indexes)
X = X[indexes]
y = y[indexes]

# WRITE ME!


# 形状確認
print(train_X.shape, train_y.shape)
print(valid_X.shape, valid_y.shape)
print(test_X.shape, test_y.shape)

ここからしばらくはテストデータを寝かしておきましょう。

## 1-3. k-NN法の実装 

ユークリッド距離を計算する関数を書いてみましょう：  

In [ ]:
# ユークリッド距離
def dist(a, b):
    # WRITE ME!
    return None

In [ ]:
# 検算
a = np.array([0, 0, 1])
b = np.array([0, 0, 1])
dist(a, b) # => 0.0

In [ ]:
c = np.array([1, 0])
d = np.array([0, 1])
dist(c, d) # => 1.4142135623730951

次に，実際にk-NN法のロジックを書いていきます。
まず、全体像を示すために骨組みだけを以下に示します。

### 骨組み (これは書き換えないでください)

In [ ]:
def classify_image(tr_X, tr_y, x, k):
  """1枚分の入力に対して
  1. 訓練データ全体との距離をそれぞれ計算
  2. 距離が短い順にソートされたインデックスを取得
  3. 2. で得たインデックスのうちk個を選択して、対応するラベルを取得する
  4. 3. で得たラベル一覧の中で最も多く出現したラベルを返す
  x: 入力画像 (D, )
  """
  pass

def predict(train_X, train_y, X, y, k):
  """k-NN法を用いて評価データを分類
  train_X: 訓練入力 (N, D)
  train_y: 訓練ラベル (N, )
  X: 評価入力 (N', D)
  y: 評価ラベル (N',)
  k: 近傍数
  """
  # ここに予測結果を格納
  pred_y = np.zeros(len(valid_X))
  
  # 各画像を分類
  for idx, x in enumerate(X):
    prediction = classify_image(train_X, train_y, x, k)
    pred_y[idx] = prediction
  
  # 正解率を計算
  score = None
  return score

# メイン処理
# 色々なkを試して、検証セットでの正解率を計算する
for k in []:
  print("k={}: validation accuracy={}".format(k, predict(train_X, train_y, valid_X, valid_y, k)))
  
best_k = 1  # ここに1番よかったkを入れる
print("k={}: test accuracy={}".format(best_k, predict(train_X, train_y, test_X, test_y, best_k)))

### Step 1: classify_imageの実装とテスト
千里の道も一歩から、大きなモデルでも小さい部分から始めれば恐れる必要はありません。  
以下のコメントに従って1つずつ書いていきましょう。

#### ヒント：
* 4. ではlabels, counts = np.unique(data, return_counts=True)を使いましょう (値とその出現数が返ってきます)

In [ ]:
def classify_image(tr_X, tr_y, eval_x, k):
  """1枚分の入力に対して
  1. 訓練データ全体との距離をそれぞれ計算
  2. 距離が短い順にソートされたインデックスを取得
  3. 2. で得たインデックスのうちk個を選択して、対応するラベルを取得する
  4. 3. で得たラベル一覧の中で最も多く出現したラベルを返す
  eval_x: 入力画像 (D, )
  """
  # 1. 
  distances = np.zeros(len(tr_X))
  for idx, x in enumerate(tr_X):
    # WRITE ME! (距離計算)
  
  # 2.
  # WRITE ME! (距離を小さい順に並び替えて，そのidxを取得→対応するラベルを出力)
  sorted_idxs = None
  
  # 3.
  # WRITE ME! 
  y_sorted = None
  
  # 4.
  # WRITE ME!
  labels, counts = None
  return None

### classify_image関数のテスト
実装した関数が正しく動いているかどうかを判定するには、まず自分で計算できる程度のサンプルを考えるのが一番です。

In [ ]:
debug_X = np.array([[0, 0], [0, 1], [1, 0], [2, 2]])
debug_y = np.array([0, 0, 1, 1])

def test_classify_image(data, k):
  return classify_image(debug_X, debug_y, np.array(data), k)

次の文を実行してエラーが出なければ上手く動いています！

In [ ]:
assert test_classify_image([0.4, 0.4], 1) == 0
assert test_classify_image([0.6, 0.4], 1) == 1
assert test_classify_image([0.6, 0.0], 3) == 0
assert test_classify_image([0.0, 0.6], 3) == 0

### 予測関数本体 (そのまま)

In [ ]:
def predict(train_X, train_y, eval_X, eval_y, k):
  """k-NN法を用いて評価データを分類
  train_X: 訓練入力 (N, D)
  train_y: 訓練ラベル (N, )
  eval_X: 評価入力 (N', D)
  eval_y: 評価ラベル (N',)
  k: 近傍数
  """
  # ここに予測結果を格納
  pred_y = np.zeros(len(eval_X))
  
  # 各画像を分類
  for idx, eval_x in enumerate(eval_X):
    prediction = classify_image(train_X, train_y, eval_x, k)
    pred_y[idx] = prediction
  
  # 正解率を計算
  score = metrics.accuracy_score(eval_y, pred_y)
  return score, pred_y

### パラメータチューニング
色々なkを試してみましょう。

In [ ]:
# メイン処理
for k in []:
  score, pred_y = predict(train_X, train_y, valid_X, valid_y, k)
  print("k={}: validation accuracy={}".format(k, score))

### 最終評価

In [ ]:
best_k = 1  # ここに1番よかったkを入れる
score, pred_y = predict(train_X, train_y, test_X, test_y, best_k)
print("k={}: test accuracy={}".format(best_k, score))

## *応用編
課題ができてしまった人はどうぞ。

* 訓練データの個数とテスト正解率の関係は？
* 混同行列を出力してみましょう、間違えたのはどの画像でしょう？ (http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html#sklearn.metrics.confusion_matrix)
* よく考えると、np.argsortで全てをソートする必要はなく、一番距離が小さいk個を選ぶだけで良い気がします。np.argpartition関数を使って余計な処理を削りましょう。
* 行列計算を工夫すると、predict関数でforループを回している部分をnumpyの計算で一気にかけます。for文を使わずに書いてみましょう。

### 予測結果の可視化

In [ ]:
visualize(test_X, pred_y, 25)

## *課題2. The Allen Mouse Brain Connectivity Atlasの解析
data/natureフォルダ中の3つのCSVファイルは、それぞれ
* nature13186-s3.csv: 蛍光強度の生データ
* nature13186-s4-w-ipsi.csv: 同側の脳領域間のW値 (結合強度)
* nature13186-s5.csv: 結合のある領域間のそれぞれの中心同士の距離

各行・列のデータ型に合わせて各ファイルをパース(解析)し、
* s3: 辞書型のリスト
* s4, s5: 脳領域名のリストとW値または強度を浮動小数点として格納したnumpy配列

として格納し、その中の要素にアクセスしてみよ。

### ヒント：
* 見出し行はif文で分岐して特別に処理する
* for内包表記を用いて2列目以降を適切に変換・処理する

### 擬似コード (pseudocode)
nature13186-s3.csvの読み込みのみ、ヒントとして擬似コードを与えます：
```
nature13186-s3.csvをオープンする
結果格納用リストを初期化
各行について
    1行目なら
        文字列をカンマで分割してリストを作成
        文字列のまま格納
    2行目なら
        文字列をカンマで分割してリストを作成
        同様に文字列のまま格納
        1行目と2行目の文字列リストを統合して、列名リストcolnamesを作成
    3行目以降の各行について、
        データ格納用のdictを作成
        文字列をカンマで分割してリストを作成
        各列の本来のデータ型に応じてintやfloatに変換
        各列名、要素について
            作ったdict[列名] <- 値
        結果格納用リストに値を格納したdictを追加

適当な行のデータを表示
```

In [ ]:
# 必ず1度実行
import os
import numpy as np

!wget  https://www.dropbox.com/s/8fqayqdai15en2f/data.zip
!unzip -n data.zip

### nature13186-s3.csv

In [ ]:
# WRITE ME!
parsed_data = []
with open("data/nature/nature13186-s3.csv") as f:
  pass

### nature13186-s4-w-ipsi.csv

In [ ]:
# WRITE ME!

### nature13186-s5.csv

In [ ]:
# WRITE ME!